In [3]:
from scipy.stats import truncnorm, multivariate_normal, norm
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
import pandas as pd

# Hyperparameteres

In [4]:
# Parameters
mu_1 = 25
mu_2 = 25
sigma_1 = 25/3
sigma_2 = 25/3
sigma_t = 25/6
iterations = 800
n_seed = 42

ax, bx = -1, 1  # The interval [ax, bx] for the uniform prior on x
sv = 1  # The variance of p(t|x)
y0 = 1  # The measurement


# GIBBS SAMPLER

In [16]:
# Helper function to sample from a truncated normal distribution
def truncated_normal(mean, std, lower, upper):
    a, b = (lower - mean) / std, (upper - mean) / std
    return truncnorm.rvs(a, b, loc=mean, scale=std)

def get_team_skills(team_A, team_B, skills_dist=skills_dist):
    # Extract mu and sigma for team_A and team_B
    mu_A, sigma_A = skills_dist[team_A]
    mu_B, sigma_B = skills_dist[team_B]
    
    return mu_A, sigma_A, mu_B, sigma_B

def set_team_skills(team_A, mu_A, sigma_A, team_B, mu_B, sigma_B, skills_dist=skills_dist):
    skills_dist[team_A] = [mu_A, sigma_A]
    skills_dist[team_B] = [mu_B, sigma_B]
    
    

def phi_value(mu_A, sigma_A, mu_B, sigma_B, sigma_t=sigma_t):
   
    denom = np.sqrt(sigma_A * sigma_A + sigma_B * sigma_B + sigma_t * sigma_t)
    # Calculate the argument for the CDF (Phi)
    argument = (mu_B - mu_A) / denom
    return norm.cdf(argument)

def predict_winner(mu_A, sigma_A, mu_B, sigma_B):
    # 1 if A wins, -1 if lose.
    return 1 if 1 - phi_value(mu_A, sigma_A, mu_B, sigma_B) >= 0.5 else -1

In [17]:
# Gibbs Sampler
def gibbs_sampler(mu_1, sigma_1, mu_2, sigma_2 ,y, iterations):
    # Initialize t
    # inital t ???????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????
    if y == 1:
        t = np.abs(np.random.randn())
    else:
        t = -np.abs(np.random.randn())

    
    samples = []
    
    for i in range(iterations):
        # p(s1,s2|t,y)
        # p(S|t,y)
        # QUESTION 3.1
        # Step 1: Draw s_1 and s_2 from the conditional distribution N(mean_s|t, cov_s|t) -
        
        
        
        sigma_s_inv = np.array([[1.0/(sigma_1 * sigma_1), 0.0],
                                [0.0, 1.0/(sigma_2 * sigma_2)]])
        
        sigma_t_s_inv = 1.0 / (sigma_t * sigma_t)
        mu_s = np.array([mu_1, mu_2]).T.reshape(2,1) # 2 x 1 (2, 1)
        A = np.array([1.0, -1.0]).reshape(1, 2) # 1 x 2
        
        # (2x2)
        cov_s_t = np.linalg.inv(sigma_s_inv + (A.T @ (sigma_t_s_inv * A) ) )
        
         # (2, )
        mean_s_t = cov_s_t @ ( (sigma_s_inv @ mu_s) + (A.T * sigma_t_s_inv * t) )
 
       
        # Draw from the multivariate normal distribution
        mean_s_t = mean_s_t.reshape(2,)
        s_1, s_2 = multivariate_normal.rvs(mean=mean_s_t, cov=cov_s_t)

        #DO WE NEED TO UPDATE: mu_1 +mu_2, sigma_1, sigma_2 ????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????

        #p(t | s_1, s_2, y)
        #p(t | S, y)

        # QUESTION 3.2
        # Step 2: Draw t from the conditional distribution 
        mean_t = s_1 - s_2 
        # print(s_1, s_2)
        if y == 1:
            t = truncated_normal(mean_t, sigma_t, 0, np.inf)  # For y = 1, t > 0
        else:
            t = truncated_normal(mean_t, sigma_t, -np.inf, 0)  # For y = -1, t < 0

        # Store samples
        samples.append((s_1, s_2, t))

    # burn_in = int(0.3 * iterations)
    burn_in = 25
    samples = np.array(samples)
    mean = np.mean(samples[burn_in:, :2], axis = 0)
    covariance = np.cov(samples, rowvar = False)
    return mean[0], mean[1], np.sqrt(covariance[0,0]), np.sqrt(covariance[1, 1])


# Run

# Moment Matching

In [ ]:
mu = mu_1 - mu_2
sigma_total_squared = sigma_t * sigma_t + sigma_1 * sigma_1 + sigma_2 * sigma_2
   
def p_t(t):
    return norm.pdf(t, loc=mu, scale=np.sqrt(sigma_total_squared))

def p_y_given_t(t, y_obs):
    if (y_obs == -1 and t < 0) or (y_obs == 1 and t > 0):
        return 1  # p(y | t ) = 1 based on your conditions
    else:
        return 0

# Function to calculate the combined probability
def p_t_given_y(t, y_obs):
    return p_t(t) * p_y_given_t(t, y_obs)

def p_t_given_y(t, mu1, mu2, sigma_t, sigma1, sigma2, y):
    
    
    # Gaussian part: N(mu1 - mu2, sqrt(sum of sigmas^2))
    gaussian_part = norm.pdf(t, loc=mu, scale=np.sqrt(sigma_total_squared))
    
    # CDF part: phi((mu2 - mu1) / sqrt(sum of sigmas^2))
    cdf_part = norm.cdf(mu / np.sqrt(sigma_total_squared))
        
    if y == 1 and t > 0: 
        return gaussian_part * (1 / (1 - cdf_part))
    elif y == -1 and t < 0: 
        return gaussian_part * (1 / cdf_part)
    else:
        return 0  # Return 0 if the condition is not met


# Calculate the combined probability
result = p_t_given_y(t_value, y_obs_value)
print(f"Combined Probability p(t={t} | y={y_obs_value}): {result:.4f}")


In [ ]:

def multiplyGauss(m1, s1, m2, s2):
    """
    Computes the Gaussian distribution N(m,s) proportional to N(m1,s1) * N(m2,s2)

    Parameters:
    m1, s1: Mean and variance of the first Gaussian
    m2, s2: Mean and variance of the second Gaussian

    Returns:
    m, s: Mean and variance of the product Gaussian
    """
    s = 1 / (1 / s1 + 1 / s2)
    m = (m1 / s1 + m2 / s2) * s
    return m, s

def divideGauss(m1, s1, m2, s2):
    """
    Computes the Gaussian distribution N(m,s) proportional to N(m1,s1) / N(m2,s2)

    Parameters:
    m1, s1: Mean and variance of the numerator Gaussian
    m2, s2: Mean and variance of the denominator Gaussian

    Returns:
    m, s: Mean and variance of the quotient Gaussian
    """
    m, s = multiplyGauss(m1, s1, m2, -s2)
    return m, s

def truncGaussMM(a, b, m0, s0):
    """
    Computes the mean and variance of a truncated Gaussian distribution

    Parameters:
    a, b: The interval [a, b] on which the Gaussian is being truncated
    m0, s0: Mean and variance of the Gaussian which is to be truncated

    Returns:
    m, s: Mean and variance of the truncated Gaussian
    """
    # Scale the interval with the mean and variance
    a_scaled, b_scaled = (a - m0) / np.sqrt(s0), (b - m0) / np.sqrt(s0)
    m = truncnorm.mean(a_scaled, b_scaled, loc=m0, scale=np.sqrt(s0))
    s = truncnorm.var(a_scaled, b_scaled, loc=m0, scale=np.sqrt(s0))
    return m, s

# Parameters

# Initialize S1, S2
mean_msg8 = 25  # Mean of message mu8
var_msg8 = 25/3 * 25/3 # Variance of message

mean_msg5 = 25  # Mean of message mu5
var_msg5 = 25/3 * 25/3 # Variance of message

y = 1 # initial y

for j in range(10):

    # Message 4 from factor f(t,s1,s2) to node t
    mean_msg4 = mean_msg8 - mean_msg5 
    var_msg4 = var_msg8 + var_msg5 + sigma_t * sigma_t

    # Do moment matching of the marginal of t
    if y == 1:
        a, b = 0, np.Inf
    else:
        a, b = np.NINF, 0

    mean_p_t_given_y, var_p_t_given_y = truncGaussMM(a, b, mean_msg4, var_msg4)

    # Compute the message from t to f(t, s1, s2)
    mean_msg12, var_msg12 = divideGauss(mean_p_t_given_y, var_p_t_given_y, mean_msg4, var_msg4)

    # Compute the message from f(t, s1, s2) to s1
    mean_msg11, var_msg11 =

    # Compute the message from f(t, s1, s2) to s2
    mean_msg10, var_msg10 =

    # Do moment matching of the marginal of s1
    mean_p_s1, var_p_s1 = truncGaussMM(ax, bx, mean_msg11, var_msg11)

    # Do moment matching of the marginal of s2
    mean_p_s2, var_p_s2 = truncGaussMM(ax, bx, mean_msg10, var_msg10)

    # Compute the updated message from s1 to f(t, s1, s2)
    mean_msg8, var_msg8 = divideGauss(mean_p_s1, var_p_s1, mean_msg11, var_msg11)
    
    # Compute the updated message from s2 to f(t, s1, s2)
    mean_msg5, var_msg5 = divideGauss(mean_p_s2, var_p_s2, mean_msg10, var_msg10)
    
    # Output result
    print(f"> s1: mean:{mean_p_s1}, var:{var_p_s1}")  
    print(f"> s2: mean:{mean_p_s2}, var:{var_p_s2}")  